In [40]:
import MySQLdb
import _mysql
import numpy as np
import csv
import pandas as pd
import csv
import os
import os

In [76]:
savedir= os.getcwd()+'/../data/data_gainloss_logfiles/'

## Participant Data

In [77]:
## Get All  Participants 
## Connect to DB
db=_mysql.connect(host="christest.c5s5wgavsyu2.us-west-2.rds.amazonaws.com",user="christest",
                  passwd="gags1313",db="chrisdb")

db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_participant'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)
header = [hh[0] for hh in header]
print(header)


['id', 'MID', 'start_date', 'start_time', 'completion_code', 'completed', 'progress', 'total_tasks', 'tasks', 'cb', 'session_id', 'AID', 'HID', 'progress_times', 'cb2', 'progress_times2', 'show_info_sheet', 'agreed_to_be_contacted', 'which_completion_message']


In [78]:
db.query("""SELECT * FROM web_experiments_participant WHERE (MID LIKE "vp%")""")
r=db.store_result() # sends the whole results
rows = r.fetch_row(100)


In [79]:
i=0
for r in rows:
    if i==0:
        df = pd.DataFrame(data=np.array((r))).T
        df.columns=header
        i+=1
    else:
        df.loc[i,]=np.array((r))
        i+=1
        

In [80]:
df.head()

,id,MID,start_date,start_time,completion_code,completed,progress,total_tasks,tasks,cb,session_id,AID,HID,progress_times,cb2,progress_times2,show_info_sheet,agreed_to_be_contacted,which_completion_message
0,1390,vp02,2017-08-22 17:20:12,1503422412,UPRI0FsHWs,0,1,1,experiment_ambicombined,0,3WH4,inperson,0,NULL,0,NULLambi combined start:2017-08-22 17:20:56.58...,0,0,0
1,1392,vp01,2017-08-22 20:04:02,1503432242,6JXd6T7gSU,0,1,1,experiment_ambicombined,0,uQ5q,inperson,0,NULL,0,NULLambi combined start:2017-08-22 20:07:19.63...,0,0,0
2,1406,vp03,2017-08-26 00:52:17,1503708737,X9KPVn7rly,0,1,1,experiment_ambicombined,0,D1Hl,inperson,0,NULL,0,NULLambi combined start:2017-08-26 00:52:22.95...,0,0,0
3,1410,vp04,2017-08-29 19:11:58,1504033918,61rFvJbmR2,0,1,1,experiment_ambicombined,0,7pM9,inperson,0,NULL,0,NULLambi combined start:2017-08-29 19:18:00.16...,0,0,0
4,1414,vp05,2017-08-30 19:45:00,1504122300,hf0a0bBmHi,0,1,1,experiment_ambicombined,0,XPAL,inperson,0,NULL,0,NULLambi combined start:2017-08-30 19:51:46.32...,0,0,0


In [81]:
df.MID.unique()

array(['vp02', 'vp01', 'vp03', 'vp04', 'vp05', 'vp06', 'vp1111', 'vp07',
       'vp003', 'vp09', 'vp10', 'vp12', 'vp11', 'vp13', 'vp14', 'vp15',
       'vp16', 'vp17', 'vp19', 'vp18', 'vp21', 'vp22', 'vp20', 'vp24',
       'vp25'], dtype=object)

## Drop Subjects if Necessary

- vp08 did not complete the online session
- vp21 (no show for shock)
- vp24 (incident during shock session)
- vp25 (can't find online data at the moment)

In [82]:
droplist = ['vp1111','vp21','vp24','vp003','vp25']
df = df[~df['MID'].isin(droplist)]

In [83]:
df.to_csv(savedir+'participant_table.csv')

## Trial Data

In [71]:
MIDs = df.MID.unique()
HIDs = np.zeros(len(MIDs))
MIDs

array(['vp02', 'vp01', 'vp03', 'vp04', 'vp05', 'vp06', 'vp07', 'vp09',
       'vp10', 'vp12', 'vp11', 'vp13', 'vp14', 'vp15', 'vp16', 'vp17',
       'vp19', 'vp18', 'vp22', 'vp20'], dtype=object)

In [72]:
db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_ambi_trial'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)
header = [hh[0] for hh in header]
print(header)

['id', 'MID', 'trial_number', 'start_date', 'trialstart', 'outcome', 'reaction_time', 'totalpoints', 'prop_left', 'prop_right', 'resp', 'revealed_left', 'revealed_right', 'AID', 'HID', 'noresp', 'session_id', 'five_trial_outcome', 'mag_left', 'mag_right', 'received_mag', 'practice', 'loss_or_reward', 'revealed_o_l', 'revealed_o_r', 'revealed_x_l', 'revealed_x_r', 'instruct_number', 'est_left_over_right']


In [73]:
## Download trial data for each participant and Save to CSV 

for MID,HID in zip(MIDs,HIDs):
    query= """SELECT * FROM web_experiments_ambi_trial WHERE (MID = '"""+str(MID)+"""') AND (HID = '"""+str(int(HID))+"""')"""
    db.query(query)
    r=db.store_result() # sends the whole results
    rows = r.fetch_row(600)
    print('MID:{0}, num rows:{1}'.format(MID,len(rows)))
    if len(rows)>0:
        
        ## Write to CSV 
        with open(savedir+MID+'_gainloss_raw.csv', 'w') as csvfile:
            writer = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(header)
            for row in rows:
                writer.writerow([x for x in row])

MID:vp02, num rows:388
MID:vp01, num rows:302
MID:vp03, num rows:367
MID:vp04, num rows:463
MID:vp05, num rows:380
MID:vp06, num rows:355
MID:vp07, num rows:383
MID:vp09, num rows:382
MID:vp10, num rows:382
MID:vp12, num rows:428
MID:vp11, num rows:382
MID:vp13, num rows:382
MID:vp14, num rows:382
MID:vp15, num rows:384
MID:vp16, num rows:382
MID:vp17, num rows:382
MID:vp19, num rows:382
MID:vp18, num rows:383
MID:vp22, num rows:382
MID:vp20, num rows:382


In [74]:
# preprocess 
for MID in MIDs:
    df = pd.read_csv(savedir+MID+'_gainloss_raw.csv')
    df=df.loc[df['practice']=='false',]
    print('MID:{0}, num rows:{1}'.format(MID,len(df)))
    df.to_csv(savedir+MID+'_gainloss_processed.csv')

MID:vp02, num rows:300
MID:vp01, num rows:300
MID:vp03, num rows:300
MID:vp04, num rows:299
MID:vp05, num rows:300
MID:vp06, num rows:300
MID:vp07, num rows:300
MID:vp09, num rows:300
MID:vp10, num rows:300
MID:vp12, num rows:300
MID:vp11, num rows:300
MID:vp13, num rows:300
MID:vp14, num rows:300
MID:vp15, num rows:300
MID:vp16, num rows:300
MID:vp17, num rows:300
MID:vp19, num rows:300
MID:vp18, num rows:300
MID:vp22, num rows:300
MID:vp20, num rows:300
